In [ ]:
import pywatershed
import pandas as pd
from pathlib import Path as pl
import json
import numpy as np
from pywatershed.parameters.prms_parameters import JSONParameterEncoder

import sys
sys.path.append('../scripts/')
from pest_utils import pars_to_tpl_entries
sys.path.append('../dependencies/')
import pyemu

#import plotly.graph_objects as go
#import plotly

## Functions
### Creates a function that writes the parameter file as a json file

In [ ]:
def write_to_json_tpl(dims, pars, json_filename):
    with open(json_filename, "w") as ofp:
        ofp.write('ptf ~\n')
        json.dump(
            {**dims,
            **pars},
            ofp,
            indent=4,
            cls=JSONParameterEncoder,
        )
    # this sucks - should be a more direct way but whatevs. it verks
    inlines = open(json_filename, 'r').readlines()
    with open(json_filename, 'w') as ofp:
        [ofp.write(i.replace('"~','~').replace('~"','~')) for i in inlines]

## We will be running on each cutout eventually, but start with one

In [ ]:
all_models = ['01473000','05431486','09112500','14015000']# Create a list of all cutouts
#rootdir = pl.Path('../NHM_extractions/20230110_pois_haj/')# Path to location of cutouts

exclude_gages = {'01473000':[],
                 '05431486':['05431022'],
                 '09112500': ['09112200'],
                 '14015000': []}
                 
c_model = all_models[1]
wkdir = pl(f'../NHM_extractions/20230110_pois_haj/{c_model}/')
c_model

## Read in the oldskool parameter file as nested dictionaries, a.k.a. a Json-style file. This file would need to be written as a .txt file like the original myparam.param, unless Joe changes the pyWatershed code to read in the json-style file instead. Check with Mike/Joe.
## Eddie and Andy changed the path for this read to the "starting values" that Parker gave us for each extraction.

In [ ]:
wkdir

In [ ]:
####This was made in a previous notebook now
# pardat = pywatershed.parameters.PrmsParameters.load(wkdir / "myparam.param")#load parameter file from extraction
# pardat.parameters_to_json(wkdir /"parameters.json")


## Now the upper portion making the jason and the lower blocks running the model should be completed prior to the output notebook..or in it.

In [ ]:
pardat = pywatershed.parameters.PrmsParameters.load_from_json(wkdir / "parameters.json")#load parameter file from extraction
pars = pardat.parameters
#pars

In [ ]:
dims = pardat.dimensions
dims

In [ ]:
#Lets check for the pars needed to run NHM model using pyWatershed

In [ ]:
nhm_processes = [
    pywatershed.PRMSSolarGeometry,
    pywatershed.PRMSAtmosphere,
    pywatershed.PRMSCanopy,
    pywatershed.PRMSSnow,
    pywatershed.PRMSRunoff,
    pywatershed.PRMSSoilzone,
    pywatershed.PRMSGroundwater,
    pywatershed.PRMSChannel,
]

nhm_params = []
for proc in nhm_processes:
    nhm_params += proc.get_parameters()

In [ ]:
#nhm_params

#### View the keys for "pars" --These will be the parameters with values listed in the "myparam_starting_vals.param" file of the extraction.

In [ ]:
pars.keys()

#### View values of one paramenter

In [ ]:
pars['nhm_id']

## Create a PEST template file version of json-style of myparam_starting_vals.param, "pars"

In [ ]:
# Make a list of hrus from "pars"
hrus = pars['nhm_id']
hrus

In [ ]:
segs = pars['nhm_seg']
segs

### Run through all the currently-defined parameters, and using Mike's function "pars_to_tpl_entries()", write param stating values to a new dataframe "par_starting_vals"

In [ ]:
# Create empty dataframe with columns parname (pestpp param name) and parval1 (pestpp starting value)
par_starting_vals = pd.DataFrame(columns=['parname','parval1', 'parubnd','parlbnd'])

In [ ]:
par_starting_vals

In [ ]:
par_starting_vals = pars_to_tpl_entries(pars, 'adjmix_rain', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'carea_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'cecn_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'emis_noppt', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'fastcoef_lin', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'freeh2o_cap', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'gwflow_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'jh_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'mann_n', hrus, segs, par_starting_vals, hru_based=False, 
                    seg_based=True, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'potet_sublim', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'rad_trncf', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'radmax', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'rain_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'slowcoef_sq', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'smidx_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'smidx_exp', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snarea_thresh', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snowinfil_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snow_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'soil2gw_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'soil_moist_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'soil_rechr_max_frac', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'ssr2gw_exp', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'ssr2gw_rate', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_allrain_offset', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_allsnow', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmin_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)


In [ ]:
par_starting_vals

In [ ]:
par_starting_vals.set_index('parname', inplace =True, drop = False)
par_starting_vals
xx = par_starting_vals.loc[par_starting_vals.parname.str.startswith('carea_max'), :]
xx

In [ ]:
# need a loop here

## Setting bounds for parameters
### There were three ways to set parameter bounds in NHM calibration:
#### 1) "not used" in the by HRU calibration, all HRU values for this type were grouped and moved as a group in the full calibration range for the parameter.
#### 2) "range" were calibrated by HRU so will move independently within the calibrations range in table 3.
#### 2b) "percent" were calibrated by HRU but only allowed a range of +/- 20% of the starting value.

In [ ]:
bnds_path = '../Supporting_information/par_cal_bounds_use.csv'
bnds = pd.read_csv(bnds_path) # Creates a data frame of the bounds for par catagories
bnds.set_index('parameter_name', inplace =True, drop = False)

In [ ]:
bnds.parameter_name.unique()

In [ ]:
##Create the lists of parameters for the claibration methods used
percent_list = bnds.loc[bnds.HRU_cal_method == 'Percent','parameter_name'].reset_index(drop = True)
range_list = bnds.loc[bnds.HRU_cal_method == 'Range','parameter_name']#.to_list() Note, all values are uniform starting values populated from the table 'par_vale_use.csv'
not_used_list = bnds.loc[bnds.HRU_cal_method == 'Not used','parameter_name']#.to_list()
print(not_used_list)

In [ ]:
#We will rewrite these using the cn, _ and .loc method edicimated to us by 'the great one.'
for idx, row in par_starting_vals.iterrows():
    for param in percent_list:
        pst_parname = str(row.parname)
        prms_parname = param
        #print(prms_parname)
        x = pst_parname.startswith(prms_parname)# Just a yes not response to if the pst parname starts with the root in "".
        if x:
            par_starting_vals.loc[pst_parname,'parubnd'] = (0.2*par_starting_vals.loc[pst_parname, 'parval1'])+par_starting_vals.loc[pst_parname, 'parval1']
            par_starting_vals.loc[pst_parname,'parlbnd'] = par_starting_vals.loc[pst_parname, 'parval1']-(0.2*par_starting_vals.loc[pst_parname, 'parval1'])

for idx, row in par_starting_vals.iterrows():
    for param in range_list:
        pst_parname = str(row.parname)
        prms_parname = param
        #print(prms_parname)
        x = pst_parname.startswith(prms_parname)# Just a yes not response to if the pst parname starts with the root in "".
        if x:
             par_starting_vals.loc[pst_parname,'parubnd'] = bnds.loc[prms_parname,'par_upper_bound']
             par_starting_vals.loc[pst_parname,'parlbnd'] = bnds.loc[prms_parname,'par_lower_bound']

#for idx, row in par_starting_vals.iterrows():
#    for param in not_used_list:
#        pst_parname = str(row.parname)
#        prms_parname = param
#        #print(prms_parname)
#        x = pst_parname.startswith(prms_parname)# Just a yes not response to if the pst parname starts with the root in "".
#        if x:
#            par_starting_vals.loc[pst_parname,'parubnd'] = (0.2*par_starting_vals.loc[pst_parname, 'parval1'])+par_starting_vals.loc[pst_parname, 'parval1']
#            par_starting_vals.loc[pst_parname,'parlbnd'] = par_starting_vals.loc[pst_parname, 'parval1']-(0.2*par_starting_vals.loc[pst_parname, 'parval1'])
#             

In [ ]:
#xx = par_starting_vals.loc[par_starting_vals.parname.str.startswith('adjmix_rain'), :]
#xx

### once we have all the parameter arrays replaced by names, we can write out the template file

In [ ]:
write_to_json_tpl(dims, pars, wkdir / 'parameters.json.tpl')
par_starting_vals.to_csv(wkdir / 'starting_par_vals.dat', index=None, sep=' ')

### Map observation name to the Instruction File (.ins)

In [ ]:
obsvals = pd.read_csv(wkdir / 'allobs.dat', delim_whitespace= True)
obsvals.set_index('obsname', inplace =True, drop = False)
#obsvals.sample(5)
print(obsvals)
print('The values for "obsval" are the true observation values.')

In [ ]:
with open(wkdir /'modelobs.dat.ins', 'w') as ofp:
    ofp.write('pif ~\n')
    ofp.write('~obsval~\n')
    [ofp.write(f'l1 w !{i}!\n') for i in obsvals.obsname]

### create PST control file object with `pyemu`

In [ ]:
pst = pyemu.Pst.from_io_files(tpl_files = [str(wkdir / 'parameters.json.tpl')],
                              in_files=[str(wkdir / 'parameters.json')],# Values for parval1 and bnds will be populated with default values
                              ins_files = [str(wkdir / 'modelobs.dat.ins')],
                              out_files = [str(wkdir / 'modelobs.dat')], #names the model output file in the control file (prior_mc.pst)--Chk with Mike
                              pst_path = '.')
#Ask Mike if pyemu reads in the values for the obs from the modelobs.dat file

## Direct editing of the PEST parameter file

## Starting parameter values
### Starting values were set from the initial parameter file used, in our case it was the "pre-calibration" values given to us by Parker. SO! No changes to those values, but we will need to customize the upper and lower bounds!

In [ ]:
pars = pst.parameter_data
#pars

In [ ]:
#pars

In [ ]:
#pars.loc['adjmix_rain:hru_5621:mon_1','parval1'] = 987236
#pst.parameter_data


### Copy parval1, upper bound and lower bound from "par_starting_vals" to pars.parval1 

In [ ]:
for idx, row in pars.iterrows():
    pars.loc[pars.parnme,'parval1'] = par_starting_vals.loc[pars.parnme,'parval1']
    pars.loc[pars.parnme,'parubnd'] = par_starting_vals.loc[pars.parnme,'parubnd']
    pars.loc[pars.parnme,'parlbnd'] = par_starting_vals.loc[pars.parnme,'parlbnd']

In [ ]:
pars.sample(50)

### Copy upper and lower bounds from par_cal_bounds_use.csv to par.parubnd and par.parlbnd
### AND...overwite parval1 with new strating values determined from default values listed in PRMS table 5.2.1 (published), https://water.usgs.gov/water-resources/software/PRMS/--Chack with jacob and make sure these jive with what they used in the cal script. NO we are not doing this anymore!

In [ ]:
prms_parnme_list =bnds['parameter_name']# Make a list of the nhm par names for loops below
#print(prms_parnme_list)

In [ ]:
#We recan delete this because we replaced this assignment above
#for idx, row in pars.iterrows():
#    for i in prms_parnme_list:
#        pst_parnme = str(row.parnme)
#        prms_parnme = prms_parnme_list[i]
#        x = pst_parnme.startswith(prms_parnme)# Just a yes not response to if the pst parname starts with the root in "".
#        if x :
#            pars.loc[pst_parnme,'parubnd'] = bnds.loc[prms_parnme,'par_upper_bound']
#            pars.loc[pst_parnme,'parlbnd'] = bnds.loc[prms_parnme,'par_lower_bound']
#            #pars.loc[pst_parnme,'parval1'] = bnds.loc[prms_parnme,'par_start_val'] remove

### we can't log transform negative parameter values

In [ ]:
pars.loc[pars.parlbnd<=0, 'partrans'] = 'none'

In [ ]:
### obs.loc[obsvals.obsname, 'obsval'] = obsvals.obsval.values

#### Set obsval in the "pst.observation_data" frame back to the true observation value

In [ ]:
obs = pst.observation_data #This pulls the "observation data" from the pst dataframe and sets it to the "obs" object (dataframe)

In [ ]:
obs.loc[obs.obsnme, :]

In [ ]:
obs.loc[obs.obsnme =='actet_mon:2000_1:5621',:] # This is the value in the modelobs.dat file?

In [ ]:
obsvals.loc[obsvals.obsname =='actet_mon:2000_1:5621',:]

In [ ]:
#obs = obs.loc[obsvals.obsname,:] #resorts datframe for easy in reading

In [ ]:
obs.loc[obsvals.obsname, 'obsval'] = obsvals.obsval.values #True observation value is copied over to obs
obs

In [ ]:
obs.loc[obs.obsnme =='actet_mon:2000_1:5621',:] # Check for change

#### Creating Groups observations

In [ ]:
obs.loc[obs.obsnme.str.startswith('l_max_actet_mon'),'obgnme'] = 'l_max_actet_mon'
obs.loc[obs.obsnme.str.startswith('g_min_actet_mon'),'obgnme'] = 'g_min_actet_mon'

obs.loc[obs.obsnme.str.startswith('l_max_actet_mean_mon'),'obgnme'] = 'l_max_actet_mean_mon'
obs.loc[obs.obsnme.str.startswith('g_min_actet_mean_mon'),'obgnme'] = 'g_min_actet_mean_mon'

obs.loc[obs.obsnme.str.startswith('l_max_recharge_ann'),'obgnme'] = 'l_max_recharge_ann'
obs.loc[obs.obsnme.str.startswith('g_min_recharge_ann'),'obgnme'] = 'g_min_recharge_ann'

obs.loc[obs.obsnme.str.startswith('l_max_soil_moist_mon'),'obgnme'] = 'l_max_soil_moist_mon'
obs.loc[obs.obsnme.str.startswith('g_min_soil_moist_mon'),'obgnme'] = 'g_min_soil_moist_mon'

obs.loc[obs.obsnme.str.startswith('l_max_soil_moist_ann'),'obgnme'] = 'l_max_soil_moist_ann'
obs.loc[obs.obsnme.str.startswith('g_min_soil_moist_ann'),'obgnme'] = 'g_min_soil_moist_ann'


#obs.loc[obs.obsnme.str.startswith('runoff_mon'),'obgnme'] = 'runoff_mon'
obs.loc[obs.obsnme.str.startswith('l_max_runoff_mon'),'obgnme'] = 'l_max_runoff_mon'
obs.loc[obs.obsnme.str.startswith('g_min_runoff_mon'),'obgnme'] = 'g_min_runoff_mon'

#obs.loc[obs.obsnme.str.startswith('sca_daily'),'obgnme'] = 'sca_daily'
obs.loc[obs.obsnme.str.startswith('l_max_sca_daily'),'obgnme'] = 'l_max_sca_daily'
obs.loc[obs.obsnme.str.startswith('g_min_sca_daily'),'obgnme'] = 'g_min_sca_daily'



#obs.loc[obs.obsnme.str.startswith('streamflow_daily'),'obgnme'] = 'streamflow_daily'

# Create EFC Groups for daily streamflows
# streamflow_daily is followed by a suffix: "efc"_"high_low" integers
# efc [1, 2, 3, 4, 5] are ['Large flood', 'Small flood', 'High flow pulse', 'Low flow', 'Extreme low flow']
# high_low [1, 2, 3] are ['Low flow', 'Ascending limb', 'Descending limb']
# Pest++ group names were written with flows in mind.

obs.loc[obs.obsnme.str.startswith('streamflow_daily_1_2'),'obgnme'] = 'streamflow_daily_large_asc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_1_3'),'obgnme'] = 'streamflow_daily_large_dsc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_2_2'),'obgnme'] = 'streamflow_daily_small_asc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_2_3'),'obgnme'] = 'streamflow_daily_small_dsc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_3_2'),'obgnme'] = 'streamflow_daily_pulse_asc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_3_3'),'obgnme'] = 'streamflow_daily_pulse_dsc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_4_1'),'obgnme'] = 'streamflow_daily_low'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_5_1'),'obgnme'] = 'streamflow_daily_exlow'

#Special group for no flow
obs.loc[obs.obsnme.str.startswith('streamflow_daily_-9999_-9999'),'obgnme'] = 'streamflow_nodata'
obs.loc[(obs.obsnme.str.startswith('streamflow_daily')) &
      (obs.obsval==-9999), 'obgnme'] = 'streamflow_nodata'
for ex in exclude_gages[c_model]:
    obs.loc[(obs.obsnme.str.startswith('streamflow_daily')) &
      (obs.obsnme.str.endswith(ex)), 'obgnme'] = 'streamflow_nodata'

obs.loc[obs.obsnme.str.startswith('streamflow_mon'),'obgnme'] = 'streamflow_mon'
obs.loc[obs.obsnme.str.startswith('streamflow_mean_mon'),'obgnme'] = 'streamflow_mean_mon'
obs.sample(30)

In [ ]:
#obs.loc[obs['obgnme']=='streamflow_nodata'].sample(30)

In [ ]:


#Set weights for groups"
## TODO: Assign weights for all but streamflow that make sense as 1/std

###Need to tailor these wts individually to the STDV values that we assume are "good."

# obs.loc[obs.obgnme=='l_max_actet_mean_mon','weight'] = 3.0E+04
# obs.loc[obs.obgnme=='g_min_actet_mean_mon','weight'] = 3.0E+04

# obs.loc[obs.obgnme=='l_max_actet_mon','weight'] = 0.75E+04
# obs.loc[obs.obgnme=='g_min_actet_mon','weight'] = 0.75E+04

# obs.loc[obs.obgnme=='l_max_recharge_ann','weight'] = 0.4E+04
# obs.loc[obs.obgnme=='g_min_recharge_ann','weight'] = 0.4E+04

# obs.loc[obs.obgnme=='l_max_soil_moist_ann','weight'] = 2.5E+03
# obs.loc[obs.obgnme=='g_min_soil_moist_ann','weight'] = 2.5E+03

# obs.loc[obs.obgnme=='l_max_soil_moist_mon','weight'] = 8E+02
# obs.loc[obs.obgnme=='g_min_soil_moist_mon','weight'] = 8E+02


# obs.loc[obs.obgnme=='l_max_sca_daily','weight'] = 0 #3E-03
# obs.loc[obs.obgnme=='g_min_sca_daily','weight'] = 0 #3E-03

# obs.loc[obs.obgnme=='l_max_runoff_mon','weight'] = 3.5
# obs.loc[obs.obgnme=='g_min_runoff_mon','weight'] = 3.5



# obs.loc[obs.obgnme.str.startswith('streamflow'), 'weight'] = \
#     10 / obs.loc[obs.obgnme.str.startswith('streamflow'),'obsval']
# obs.loc[obs.obgnme=='streamflow_nodata','weight'] = 0

# # special case for streamflow with 0 observed value
# obs.loc[(obs.obsval<=1) & (obs.obgnme.str.startswith('stream')), 'weight'] = 1.0



In [ ]:
obs.loc[(obs.obsval<=1) & (obs.obgnme.str.startswith('stream'))]

In [ ]:
#obs.loc[obs.obgnme.str.startswith('streamflow')

## now we flip these weights back to standard deviation for the noise ensemble and then do not revisit STD, although we will adjust weights to rebalance PHI--Retooled

In [ ]:
#obs.loc[:,'standard_deviation'] = [1/w if w!=0 else 1e-6 for w in obs.weight]

## Set SD and bounds for obs from file "Observation_standard_deviation.csv" in Supporting Information folder; if you want to change bounds and SD, change values in the .csv file. Primarily to make sure values during the prior don't go negative.

In [ ]:
obs_sdbnds_path = '../Supporting_information/Observation_standard_deviation.csv'
obs_sdbnds = pd.read_csv(obs_sdbnds_path) # Creates a data frame of the bounds for par catagories
obs_sdbnds.set_index('obsgroup', inplace =True, drop = False)

In [ ]:
obs_sdbnds

In [ ]:
obs_sdbnds.index =[i.strip() for i in obs_sdbnds.index]# strip removes the extra spaces and /n etc

In [ ]:
obs_sdbnds.index.unique()

In [ ]:
obs['lower_bound'] = 0
obs['upper_bound'] = np.nan
obs['standard_deviation'] = np.nan
#obs['weight'] = np.nan

In [ ]:
obsgroup_list = obs_sdbnds['obsgroup']
obsgroup_list

In [ ]:
obs_sdbnds.columns

In [ ]:
obs.loc[obs.obgnme=='streamflow_nodata']

In [ ]:
obs

In [ ]:
for cn,_ in obs.groupby('obgnme'):
    obs.loc[obs.obgnme == cn, 'upper_bound']= obs_sdbnds.loc[cn, 'obsubnd']
    #print(cn)

In [ ]:
for cn,_ in obs.groupby('obgnme'):
    obs_group_percent = obs_sdbnds.loc[cn, 'noise_percent']
    obs.loc[obs.obgnme == cn, 'standard_deviation']= obs_group_percent*(obs.loc[obs.obgnme==cn, 'obsval'])
    #print(cn)

#Replace std value with 9999 where obsval values with "9999"
obs.loc[obs.obsval == -9999, 'standard_deviation']= 9999

In [ ]:
#obs.loc[obs.standard_deviation.isnull()]


In [ ]:
#But, to read in the "other" SD, the SD for the value, not the noise.

In [ ]:

#Do this for streamflow but not the rest
for cn,_ in obs.groupby('obgnme'):
    if cn.startswith('streamflow_'):
        obs_group_percent = obs_sdbnds.loc[cn, 'wt_percent']#"wt_percent" in th etable is a fractional value from csv
        obs.loc[obs.obgnme == cn, 'weight']= 1/(obs_group_percent*(obs.loc[obs.obgnme==cn, 'obsval']))
    else:
        obs_group_percent = obs_sdbnds.loc[cn, 'wt_percent']
        obs.loc[obs.obgnme == cn, 'weight']= 1/obs_group_percent


#For the inequality calibration obs, do NOT take weight calc using the obs val
obs.loc[obs.obgnme.str.startswith('streamflow_'),'weight'] = 'streamflow_daily_large_asc'


In [ ]:
obs.loc[obs.obgnme=='l_max_sca_daily','weight'] = 10#3E-03
obs.loc[obs.obgnme=='g_min_sca_daily','weight'] = 10#3E-00

obs.loc[obs.obgnme.str.startswith('streamflow'), 'weight'] = \
    10 / obs.loc[obs.obgnme.str.startswith('streamflow'),'obsval']

obs.loc[obs.obgnme=='streamflow_nodata','weight'] = 0

# special case for streamflow with 0 observed value
obs.loc[(obs.obsval<=1) & (obs.obgnme.str.startswith('stream')), 'weight'] = 1.0

#Replace -9999 obs_val values with 0 weight
obs.loc[obs.obsval == -9999, 'weight']= 0

In [ ]:
obs.sample(50)

# consolidate the run scripts into a single script
### Eddie commented out after modification of the forward_run.py file with James during debugging. Eddie will eventually fix this and bring it back in.

In [ ]:
imports = [i.strip() for i in open('../scripts/run-pynhm.py', 'r').readlines() if i.strip().startswith('import')]
imports.extend([i.strip() for i in open('../scripts/post-process_model_output.py', 'r').readlines() if i.strip().startswith('import')])

runbiz = [i.rstrip() for i in open('../scripts/run-pynhm.py', 'r').readlines() if not i.strip().startswith('import')]
runbiz.append('print("#### RUN DONE, TIME TO POSTPROCESS ####")')
runbiz.extend([i.rstrip() for i in open('../scripts/post-process_model_output.py', 'r').readlines() if not i.strip().startswith('import')])


In [ ]:
#runbiz

In [ ]:
# dedupe the imports
imports = list(set(imports))


### now write out all the forward run stuff

In [ ]:
with open(wkdir / 'forward_run.py', 'w') as ofp:
    [ofp.write(f'{line}\n') for line in imports+runbiz]

### and set the consolidated forward_run.py file to the pst object

In [ ]:
pst.model_command = ['python forward_run.py']

In [ ]:
pst.control_data.noptmax=0 #or -1 later, 0 at first

### set some PEST++ specific parmeters

In [ ]:
pst.pestpp_options["ies_num_reals"] = 500  

pst.pestpp_options["ies_bad_phi_sigma"] = 2.5
pst.pestpp_options["overdue_giveup_fac"] = 4
pst.pestpp_options["ies_no_noise"] = False
pst.pestpp_options["ies_drop_conflicts"] = False
pst.pestpp_options["ies_pdc_sigma_distance"] = 3.0
pst.pestpp_options['ies_autoadaloc']=False
pst.pestpp_options['ies_num_threads']=4
pst.pestpp_options['ies_lambda_mults']=(0.1,1.0,10.0,100.0)
pst.pestpp_options['lambda_scale_fac'] = (0.75,0.9,1.0,1.1)
pst.pestpp_options['ies_subset_size']=10

# set SVD for some regularization
pst.svd_data.maxsing = 250

In [ ]:
assert len(pst.observation_data.loc[pst.observation_data.weight==0]) >0

In [ ]:
pst.parameter_data=pst.parameter_data[['parnme','partrans',	'parchglim', 'parval1',	'parlbnd', 'parubnd','pargp','scale', 'offset', 'dercom']]

### special case for just this one value with busted bounds 

In [ ]:
#pst.parameter_data.loc['smidx_exp:hru_84017']

In [ ]:
if 'smidx_exp:hru_84017' in pst.parameter_data.index:
    pst.parameter_data.loc['smidx_exp:hru_84017', 'parval1'] = 0.003
    pst.parameter_data.loc['smidx_exp:hru_84017', 'parubnd'] = .003*2
    

In [ ]:
pst.write(str(wkdir / 'prior_mc.pst'), version=2)

In [ ]:
#[pst.observation_data[i].isnull().unique() for i in pst.observation_data.columns]
obs.loc[obs.weight.isnull()].obgnme.unique()

In [ ]:
obs.isnull().values.any()

In [ ]:
len(pst.observation_data), len(pst.observation_data.dropna())

In [ ]:
pst.observation_data.loc[list(set(pst.observation_data.index) - set(pst.observation_data.dropna().index))]

In [ ]:
pst.observation_data.loc[(pst.observation_data.obgnme=='streamflow_daily_exlow') &
(pst.observation_data.obsval==0)]

In [ ]:
pst.observation_data.loc[(pst.observation_data.obsnme=='streamflow_daily_3_2:2000_7_11:05431022')] #&
#(pst.observation_data.weight>0)]